In [ ]:
import pandas as pd
import sys
sys.path.append("..")
import pandas as pd
from pathlib import Path
import numpy as np
import pickle
from sif_src.glove import load_glove_vectors, calculate_glove_corpus_embeddings
from sif_src.sif import flatten, compute_sif_embeddings_queries, compute_sif_embeddings_texts, compute_word_frequencies, compute_sif_weights, remove_pc_sif
import tqdm
from collections import defaultdict, Counter
from collections.abc import Iterable
from sklearn.decomposition import TruncatedSVD


Calculate and pickle both embeddings for training set

In [ ]:
train_df = pd.read_pickle("../pickle_backups/marco_train_df2024-06-04T17.38.1717490321.pickle")
train_passages = train_df["passages"]
train_texts = [passage["passage_text"] for passage in train_passages]
train_df["texts"] = train_texts

#homogenize MS MARCO columns
train_df['query'] = train_df['query'].apply(lambda x: [x])

# Load GloVe vectors
glove_vectors = load_glove_vectors('../wv/glove.6B.50d.txt')


In [ ]:
queries = train_df["query"]
texts = train_df["texts"]
word_freq = compute_word_frequencies(queries + texts)

compute SIF embeddings

In [ ]:
sif_weights = compute_sif_weights(word_freq)

In [ ]:
query_sif_embeddings = compute_sif_embeddings_queries(queries, glove_vectors, sif_weights)
texts_sif_embeddings = compute_sif_embeddings_texts(texts, glove_vectors, sif_weights)

In [ ]:
texts_sif_embeddings = remove_pc_sif(texts_sif_embeddings, alpha=0.001)

In [ ]:
train_df["sif_queries"] = list(query_sif_embeddings)
train_df["sif_texts"] = list(texts_sif_embeddings)

In [ ]:
#queries = train_df["query"]
#texts = train_df["texts"]

query_glove_embeddings = calculate_glove_corpus_embeddings(queries, glove_vectors)


In [ ]:
texts_glove_embeddings = calculate_glove_corpus_embeddings(texts, glove_vectors)

In [ ]:
train_df["glove_queries"] = list(query_glove_embeddings)
train_df["glove_texts"] = list(texts_glove_embeddings)

In [ ]:
import pickle
from datetime import datetime

TIMESTAMP = datetime.now()
OUTPUT_PATH = Path("../pickle_backups")
TIMESTAMP_STR = TIMESTAMP.isoformat().replace(":", ".")

with open(OUTPUT_PATH / 
    f"train_embeddings{TIMESTAMP.strftime(r'%Y-%m-%dT%H.%M.%s')}.pickle",
    "wb"
) as f:
    pickle.dump(train_df, f)